In [26]:
from google.colab import drive
drive.mount('/content/drive')
from sklearn.model_selection import KFold
import keras
import scipy.io as sci
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, accuracy_score
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from keras.callbacks import LearningRateScheduler
from keras.callbacks import History
from sklearn.preprocessing import MinMaxScaler
Scaler = MinMaxScaler()
import warnings
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
import seaborn as sns

from sklearn import svm
from sklearn.linear_model import LogisticRegression
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
matData = sci.loadmat('/content/drive/My Drive/final_project_mat.mat')

In [3]:
df = pd.DataFrame({'X': matData['sailing_data'][0], 'Y': matData['sailing_data'][0]})

In [5]:
plt.figure(figsize = (10,5))
df.X[0].append(df.X[0],df.Y[0][1],axis = 1)
ddd=pd.DataFrame(df.X[0])

sns.heatmap(ddd.corr(),annot = True, cmap = 'rainbow')
plt.show()


AttributeError: 'numpy.ndarray' object has no attribute 'append'

<Figure size 1000x500 with 0 Axes>

In [12]:
df.Y[1].shape

(135, 1)

In [ ]:
error = np.array([])
rmse = np.array([])
for t in range(1,124):
  X_train,Y_train = [],[]
  # LOOCV
  train_subjects = [i for i in range(1,124) if i!=t]
  for j in train_subjects:
    X_train = np.append(X_train,df.X[0][j-1].T,axis = 0)
    Y_train = np.append(Y_train,df.Y[1][j-1].T,axis = 0)
  X_test = df.X[0][t-1].T
  Y_test = df.Y[1][t-1].T

  X_train = X_train.reshape(122,8)
  Y_train = Y_train.reshape(122,1)
  X_test = X_test.reshape(1,8)
  Y_test = Y_test.reshape(1,1)
  clf = LogisticRegression(penalty = 'l2',random_state = 0,solver = 'liblinear').fit(X_train,Y_train)
  Y_predicted = clf.predict(X_test)
  print('Iteration:',t)
  print( np.mean(Y_test - Y_predicted))
  error = np.append(error,[np.mean(Y_test-Y_predicted)],axis = 0)
  rmse = np.append(rmse,[np.sqrt(np.mean((Y_test-Y_predicted)**2))],axis = 0)


In [18]:
sum(error)

3.0

In [28]:
import statistics

In [35]:
#Tuning Hyperparameters
parameters = df.X[0]
target = df.Y[1]
x_train, x_test, y_train, y_test = train_test_split(parameters, target, test_size=0.2)
mean_vector = []
variance_vec = []
for i in range(1,12):
  mean_vector = np.append(mean_vector,[statistics.mean(x_train[:,i-1])],axis = 0)
  variance_vec = np.append(variance_vec,[statistics.variance(x_train[:,i-1])],axis = 0)
  x_train[:,i-1] = (x_train[:,i-1]-(mean_vector[i-1]))/variance_vec[i-1]
  x_test[:,i-1]  =  (x_test[:,i-1]-(mean_vector[i-1]))/variance_vec[i-1]


In [5]:
reg_models = [LogisticRegression(penalty = 'l2',random_state = 0,solver = 'liblinear'),LogisticRegression(penalty = 'l2',random_state = 0,solver = 'lbfgs'),LogisticRegression(penalty = 'l2',random_state = 0,solver = 'sag'),
             LogisticRegression(penalty = 'l1',random_state = 0,solver = 'liblinear'),LogisticRegression(penalty = 'none',random_state = 0,solver = 'lbfgs'),LogisticRegression(penalty = 'none',random_state = 0,solver = 'sag'),
             LogisticRegression(penalty = 'l2',random_state = 0,solver = 'liblinear',C = 0.001),LogisticRegression(penalty = 'l2',random_state = 0,solver = 'lbfgs',C = 0.001),LogisticRegression(penalty = 'l2',random_state = 0,solver = 'sag',C = 0.001),
             LogisticRegression(penalty = 'l1',random_state = 0,solver = 'liblinear',C = 0.001),LogisticRegression(penalty = 'none',random_state = 0,solver = 'lbfgs',C = 0.001),LogisticRegression(penalty = 'none',random_state = 0,solver = 'sag',C = 0.001),
             LogisticRegression(penalty = 'l2',random_state = 0,solver = 'liblinear',C = 0.01),LogisticRegression(penalty = 'l2',random_state = 0,solver = 'lbfgs',C = 0.01),LogisticRegression(penalty = 'l2',random_state = 0,solver = 'sag',C = 0.01),
             LogisticRegression(penalty = 'l1',random_state = 0,solver = 'liblinear',C = 0.01),LogisticRegression(penalty = 'none',random_state = 0,solver = 'lbfgs',C = 0.01),LogisticRegression(penalty = 'none',random_state = 0,solver = 'sag',C = 0.01),LogisticRegression(penalty = 'l2',random_state = 0,solver = 'liblinear',C = 0.1),LogisticRegression(penalty = 'l2',random_state = 0,solver = 'lbfgs',C = 0.1),LogisticRegression(penalty = 'l2',random_state = 0,solver = 'sag',C = 0.1),
             LogisticRegression(penalty = 'l1',random_state = 0,solver = 'liblinear',C = 0.1),LogisticRegression(penalty = 'none',random_state = 0,solver = 'lbfgs',C = 0.1),LogisticRegression(penalty = 'none',random_state = 0,solver = 'sag',C = 0.1)]

In [6]:
from prettytable import PrettyTable


In [7]:
hyperparameter_score_list_1 = []
i = 0
for clf in reg_models:
  i = i+1
  scores = cross_validate(clf, x_train, y_train, cv=10, scoring='accuracy')
  mean_score = np.mean(scores['test_score'])
  hyperparameter_score_list_1.append([i, mean_score])

In [8]:
myTable = PrettyTable(["model", "Avg accuracy"])
for row in hyperparameter_score_list_1:
    myTable.add_row([row[0], round(row[1],3)])
print(myTable)

+-------+--------------+
| model | Avg accuracy |
+-------+--------------+
|   1   |    0.933     |
|   2   |    0.934     |
|   3   |    0.938     |
|   4   |    0.933     |
|   5   |    0.934     |
|   6   |    0.933     |
|   7   |    0.934     |
|   8   |    0.934     |
|   9   |    0.934     |
|   10  |    0.934     |
|   11  |    0.934     |
|   12  |    0.933     |
|   13  |    0.934     |
|   14  |    0.943     |
|   15  |    0.929     |
|   16  |    0.934     |
|   17  |    0.934     |
|   18  |    0.933     |
|   19  |    0.938     |
|   20  |    0.934     |
|   21  |    0.933     |
|   22  |    0.934     |
|   23  |    0.934     |
|   24  |    0.933     |
+-------+--------------+


In [32]:
hyperparameter_score_list_2 = []
i = 0
for clf in reg_models:
  i = i+1
  scores = cross_validate(clf, x_train, y_train, cv=10, scoring='neg_log_loss')
  mean_score = np.mean(scores['test_score'])
  hyperparameter_score_list_2.append([i, mean_score])

In [ ]:
myTable2 = PrettyTable(["model", "Avg neg_log_loss"])
for row in hyperparameter_score_list_2:
    myTable2.add_row([row[0], round(row[1],3)])
print(myTable2)

In [9]:
hyperparameter_score_list_3 = []
i = 0
for clf in reg_models:
  i = i+1
  scores = cross_validate(clf, x_train, y_train, cv=10, scoring='f1')
  mean_score = np.mean(scores['test_score'])
  hyperparameter_score_list_3.append([i, mean_score])

In [10]:
myTable3 = PrettyTable(["model", "Avg f1_score"])
for row in hyperparameter_score_list_3:
    myTable3.add_row([row[0], round(row[1],3)])
print(myTable3)

+-------+--------------+
| model | Avg f1_score |
+-------+--------------+
|   1   |    0.283     |
|   2   |     0.33     |
|   3   |    0.283     |
|   4   |    0.217     |
|   5   |     0.33     |
|   6   |    0.283     |
|   7   |     0.0      |
|   8   |     0.0      |
|   9   |     0.0      |
|   10  |     0.0      |
|   11  |     0.33     |
|   12  |    0.283     |
|   13  |     0.0      |
|   14  |     0.3      |
|   15  |    0.067     |
|   16  |     0.0      |
|   17  |     0.33     |
|   18  |    0.283     |
|   19  |    0.233     |
|   20  |    0.263     |
|   21  |    0.217     |
|   22  |     0.0      |
|   23  |     0.33     |
|   24  |    0.283     |
+-------+--------------+


In [34]:
hyperparameter_score_list_4 = []
i = 0
for clf in reg_models:
  i = i+1
  scores = cross_validate(clf, x_train, y_train, cv=10, scoring='recall')
  mean_score = np.mean(scores['test_score'])
  hyperparameter_score_list_4.append([i, mean_score])

In [ ]:
myTable4 = PrettyTable(["model", "recall"])
for row in hyperparameter_score_list_4:
    myTable4.add_row([row[0], round(row[1],3)])
print(myTable4)

In [35]:
hyperparameter_score_list_5 = []
i = 0
for clf in reg_models:
  i = i+1
  scores = cross_validate(clf, x_train, y_train, cv=10, scoring='precision')
  mean_score = np.mean(scores['test_score'])
  hyperparameter_score_list_5.append([i, mean_score])

In [ ]:
myTable5 = PrettyTable(["model", "precision"])
for row in hyperparameter_score_list_5:
    myTable5.add_row([row[0], round(row[1],3)])
print(myTable5)

In [36]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, auc,roc_curve

In [37]:
clf_columns = []
clf_compare = pd.DataFrame(columns = clf_columns)

row_index = 0
for alg in reg_models:

    predicted = alg.fit(x_train, y_train).predict(x_test)
    fp, tp, th = roc_curve(y_test, predicted)
    clf_name = alg.__class__.__name__
    clf_compare.loc[row_index, 'Train Accuracy'] = round(alg.score(x_train, y_train), 5)
    clf_compare.loc[row_index, 'Test Accuracy'] = round(alg.score(x_test, y_test), 5)
    clf_compare.loc[row_index, 'Precission'] = round(precision_score(y_test, predicted),5)
    clf_compare.loc[row_index, 'Recall'] = round(recall_score(y_test, predicted),5)
    clf_compare.loc[row_index, 'AUC'] = round(auc(fp, tp),5)

    row_index+=1

clf_compare.sort_values(by = ['Test Accuracy'], ascending = False, inplace = True)
print(clf_compare)

    Train Accuracy  Test Accuracy  Precission  Recall  AUC
12         0.93878           1.00         0.0     0.0  NaN
21         0.93878           1.00         0.0     0.0  NaN
6          0.93878           1.00         0.0     0.0  NaN
7          0.93878           1.00         0.0     0.0  NaN
8          0.93878           1.00         0.0     0.0  NaN
9          0.93878           1.00         0.0     0.0  NaN
15         0.93878           1.00         0.0     0.0  NaN
14         0.94898           1.00         0.0     0.0  NaN
13         0.94898           0.96         0.0     0.0  NaN
20         0.94898           0.96         0.0     0.0  NaN
19         0.95918           0.96         0.0     0.0  NaN
18         0.94898           0.96         0.0     0.0  NaN
17         0.95918           0.96         0.0     0.0  NaN
0          0.95918           0.96         0.0     0.0  NaN
11         0.95918           0.96         0.0     0.0  NaN
5          0.95918           0.96         0.0     0.0  N

In [12]:
from sklearn.metrics import classification_report, confusion_matrix


In [36]:
clf = LogisticRegression(penalty = 'l2',random_state = 0,solver = 'liblinear',class_weight = {0:0.1,1:1})
clf.fit(x_train,y_train)
predicted=clf.predict(x_test)
print(classification_report(y_test, predicted))
print(confusion_matrix(y_test,predicted))
print(clf.coef_)

              precision    recall  f1-score   support

           0       0.97      0.67      0.79        48
           1       0.20      0.80      0.32         5

    accuracy                           0.68        53
   macro avg       0.58      0.73      0.56        53
weighted avg       0.90      0.68      0.75        53

[[32 16]
 [ 1  4]]
[[-0.10812495 -0.00128428  0.00336278  0.          0.          0.00027563
   0.          0.          0.0098761   0.          0.        ]]
